## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database.ipynb/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Lincoln,AU,-34.7333,135.8667,57.92,62,77,12.28,broken clouds
1,1,Mataura,NZ,-46.1927,168.8643,44.47,91,83,3.11,broken clouds
2,2,Norrtalje,SE,59.7580,18.7050,51.03,73,36,1.63,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,33.12,86,19,13.20,few clouds
4,4,Upernavik,GL,72.7868,-56.1549,19.92,59,0,4.05,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Souillac,MU,76.35,few clouds,-20.5167,57.5167,
9,Saint-Pierre,RE,80.29,clear sky,-21.3393,55.4781,
15,Coahuayana,MX,77.97,overcast clouds,18.7333,-103.6833,
19,Butaritari,KI,83.26,scattered clouds,3.0707,172.7902,
22,Hithadhoo,MV,82.20,overcast clouds,-0.6000,73.0833,
24,Cabo San Lucas,MX,85.51,few clouds,22.8909,-109.9124,
25,Bull Savanna,JM,76.78,clear sky,17.8868,-77.5902,
32,Bambous Virieux,MU,76.21,few clouds,-20.3428,57.7575,
37,Labuan,MY,89.76,scattered clouds,5.2767,115.2417,
38,Daru,PG,78.85,broken clouds,-9.0763,143.2092,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Souillac,MU,76.35,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
9,Saint-Pierre,RE,80.29,clear sky,-21.3393,55.4781,Lindsey Hôtel
15,Coahuayana,MX,77.97,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
19,Butaritari,KI,83.26,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
22,Hithadhoo,MV,82.20,overcast clouds,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...,...,...
551,Namatanai,PG,83.14,light rain,-3.6667,152.4333,Seagulls Inn Namatani
552,Chake Chake,TZ,80.91,light rain,-5.2459,39.7666,Hotel Archipelago
556,Do Gonbadan,IR,86.92,clear sky,30.3586,50.7981,شبکه بهداشت و درمان گچساران
557,Cozumel,MX,80.58,scattered clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] !='']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Souillac,MU,76.35,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
9,Saint-Pierre,RE,80.29,clear sky,-21.3393,55.4781,Lindsey Hôtel
15,Coahuayana,MX,77.97,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
19,Butaritari,KI,83.26,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
22,Hithadhoo,MV,82.20,overcast clouds,-0.6000,73.0833,Scoop Guest House
24,Cabo San Lucas,MX,85.51,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
25,Bull Savanna,JM,76.78,clear sky,17.8868,-77.5902,Big Red's
32,Bambous Virieux,MU,76.21,few clouds,-20.3428,57.7575,Casa Tia Villa
37,Labuan,MY,89.76,scattered clouds,5.2767,115.2417,Tiara Labuan Hotel
38,Daru,PG,78.85,broken clouds,-9.0763,143.2092,Daru Lodge


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(28.0, 29.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=290, point_radius=3.5)

In [23]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))